# QuadCopter Hovercraft Trajectory 

In [1]:
using JuMP
using Ipopt,AmplNLWriter, NLopt, Gurobi, Mosek

## Hovercraft Problem Data 

In [2]:
k = 3             # number of waypoints
T = zeros(Int, k) # vector of timepoints
xw = zeros(3,k)   # vector of waypoint locations

T[1] = 1
T[2] = 3
T[3] = 5

# define waypoints from origin (0,0,0) to a height of 1 meter (0,0,1)
xw = [ 0 0   0
       0 0.5 1
       0 1   1];

## Quadcopter Model Data 

In [3]:
# gravitational acceleration
g = 9.81

# mass of quadcopter
mass = 0.468

# distance between rotor and center of mass of quadcopter
l = 0.225

# lift constant
K = 2.980*10.0^-6

# drag constant
b = 1.140*10.0^-7

# Inertial moment of the rotor
I_M = 3.357*10.0^-5

# Drag force coeffecients
Ax = 0.25
Ay = 0.25
Az = 0.25

# Quadcopter Inertia
Ixx = 4.856*10.0^-3
Iyy = 4.856*10.0^-3
Izz = 8.801*10.0^-3
;

## Quadcopter Mathematical Model 

In [4]:
# Commented to make it simple
#function Sin(x)
#    return sin(x)
#end

#function Cos(x)
#    return cos(x)
#end

#function Tan(x)
#    return tan(x)
#end

#JuMP.register(:S, 1, Sin, autodiff=true)
#JuMP.register(:C, 1, Cos, autodiff=true)
#JuMP.register(:T, 1, Tan, autodiff=true);

In [5]:
#m = Model(solver=IpoptSolver())
m = Model(solver=AmplNLSolver("bonmin"))
#m = Model(solver=NLoptSolver(algorithm=:LD_MMA))
#m = Model(solver=GurobiSolver())
#m = Model(solver=MosekSolver())

    # postion [x,y,z] in inertial frame
@variable(m, ξ[1:3, 1:T[k]])
@NLexpression(m, x[i=1:T[k]], ξ[1,i])
@NLexpression(m, y[i=1:T[k]], ξ[2,i])
@NLexpression(m, z[i=1:T[k]], ξ[3,i])

# attitude [ϕ,θ,ψ] in inertial frame
@variable(m, η[1:3, 1:T[k]])
@NLexpression(m, ϕ[i=1:T[k]], η[1,i])
@NLexpression(m, θ[i=1:T[k]], η[2,i])
@NLexpression(m, ψ[i=1:T[k]], η[3,i])

# linear velocities in the inertial frame
@variable(m, -1 <= LVi[1:3, 1:T[k]] <= 1)

# linear acceleration in the inertial frame
@variable(m, -2 <= LAi[1:3, 1:T[k]] <= 2)

# angular acceleration in the inertial frame
@variable(m, -30 <= AAi[1:3, 1:T[k]] <= 30)

# angular velocities in the inertial frame
@variable(m, -30 <= AVi[1:3, 1:T[k]] <= 30)

# linear velocities in the body frame
@variable(m, -30 <= LVb[1:3, 1:T[k]] <= 30)

# linear acceleration in the body frame
@variable(m, -30 <= LAb[1:3, 1:T[k]] <= 30)

# angular acceleration in the body frame
@variable(m, -30 <= AAb[1:3, 1:T[k]] <= 30)

# angular velocities in the body frame
@variable(m, -30 <= AVb[1:3, 1:T[k]] <= 30)
@NLexpression(m, p[i=1:T[k]], AVb[1,i])
@NLexpression(m, q[i=1:T[k]], AVb[2,i])
@NLexpression(m, r[i=1:T[k]], AVb[3,i])

# Angular velocity of rotor
@variable(m, 0 <= ωSq[1:4, 1:T[k]] <= 10)

# Rotor force generated in quad's frame of reference
@NLexpression(m, f1[i=1:T[k]], K*ωSq[1,i])
@NLexpression(m, f2[i=1:T[k]], K*ωSq[2,i])
@NLexpression(m, f3[i=1:T[k]], K*ωSq[3,i])
@NLexpression(m, f4[i=1:T[k]], K*ωSq[4,i])

# Thrust in body's frame of reference
@NLexpression(m, TB3[i=1:T[k]], f1[i] + f2[i] + f3[i] + f4[i])

# Rotor force generated in quad's frame of reference
@NLexpression(m, τM1[i=1:T[k]], b*ωSq[1,i])
@NLexpression(m, τM2[i=1:T[k]], b*ωSq[2,i])
@NLexpression(m, τM3[i=1:T[k]], b*ωSq[3,i])
@NLexpression(m, τM4[i=1:T[k]], b*ωSq[4,i])

# Torque in body's frame of reference
@NLexpression(m, τbϕ[i=1:T[k]] , l*K*(-ωSq[2,i] + ωSq[4,i]))
@NLexpression(m, τbθ[i=1:T[k]] , l*K*(-ωSq[1,i] + ωSq[3,i]))
@NLexpression(m, τbψ[i=1:T[k]] , τM1[i] + τM2[i] + τM3[i] + τM4[i])
;

In [6]:
# Rotation Matrix from Inertial to the Body Frame
@NLexpression(m, R11[i=1:T[k]], cos(ψ[i])*cos(θ[i]))
@NLexpression(m, R12[i=1:T[k]], cos(ψ[i])*sin(θ[i])*sin(ϕ[i]) - sin(ψ[i])*cos(ϕ[i]))
@NLexpression(m, R13[i=1:T[k]], cos(ψ[i])*sin(θ[i])*cos(ϕ[i]) + sin(ψ[i])*sin(ϕ[i]))
@NLexpression(m, R21[i=1:T[k]], sin(ψ[i])*cos(θ[i]))
@NLexpression(m, R22[i=1:T[k]], sin(ψ[i])*sin(θ[i])*sin(ϕ[i]) + cos(ψ[i])*cos(ϕ[i]))
@NLexpression(m, R23[i=1:T[k]], sin(ψ[i])*sin(θ[i])*cos(ϕ[i]) - cos(ψ[i])*sin(ϕ[i]))
@NLexpression(m, R31[i=1:T[k]], -sin(θ[i]))
@NLexpression(m, R32[i=1:T[k]], cos(θ[i])*sin(ϕ[i]))
@NLexpression(m, R33[i=1:T[k]], cos(θ[i])*cos(ϕ[i]))

# Simplified
#@NLexpression(m, R11[i=1:T[k]], 1)
#@NLexpression(m, R12[i=1:T[k]], (θ[i])*(ϕ[i]) - (ψ[i]))
#@NLexpression(m, R13[i=1:T[k]], (θ[i]) + (ψ[i])*(ϕ[i]))
#@NLexpression(m, R21[i=1:T[k]], (ψ[i]))
#@NLexpression(m, R22[i=1:T[k]], (ψ[i])*(θ[i])*(ϕ[i]) + 1)
#@NLexpression(m, R23[i=1:T[k]], (ψ[i])*(θ[i]) - (ϕ[i]))
#@NLexpression(m, R31[i=1:T[k]], -(θ[i]))
#@NLexpression(m, R32[i=1:T[k]], (ϕ[i]))
#@NLexpression(m, R33[i=1:T[k]], 1)
;

In [7]:
# Transformation matrix for angular velocities from inertial frame to body frame
@NLexpression(m, W11[i=1:T[k]], 1)
@NLexpression(m, W12[i=1:T[k]], 0)
@NLexpression(m, W13[i=1:T[k]], -sin(θ[i]))
@NLexpression(m, W21[i=1:T[k]], 0)
@NLexpression(m, W22[i=1:T[k]], cos(ϕ[i]))
@NLexpression(m, W23[i=1:T[k]], cos(θ[i])*sin(ϕ[i]))
@NLexpression(m, W31[i=1:T[k]], 0)
@NLexpression(m, W32[i=1:T[k]], -sin(ϕ[i]))
@NLexpression(m, W33[i=1:T[k]], cos(θ[i])*cos(ϕ[i]))
# Simplified
#@NLexpression(m, W11[i=1:T[k]], 1)
#@NLexpression(m, W12[i=1:T[k]], 0)
#@NLexpression(m, W13[i=1:T[k]], -(θ[i]))
#@NLexpression(m, W21[i=1:T[k]], 0)
#@NLexpression(m, W22[i=1:T[k]], 1)
#@NLexpression(m, W23[i=1:T[k]], (ϕ[i]))
#@NLexpression(m, W31[i=1:T[k]], 0)
#@NLexpression(m, W32[i=1:T[k]], -(ϕ[i]))
#@NLexpression(m, W33[i=1:T[k]], 1)

# Transformation matrix for angular velocities from body frame to inertial frame
@NLexpression(m, Winv11[i=1:T[k]], 1)
@NLexpression(m, Winv12[i=1:T[k]], sin(ϕ[i])*tan(θ[i]))
@NLexpression(m, Winv13[i=1:T[k]], cos(ϕ[i])*tan(θ[i]))
@NLexpression(m, Winv21[i=1:T[k]], 0)
@NLexpression(m, Winv22[i=1:T[k]], cos(ϕ[i]))
@NLexpression(m, Winv23[i=1:T[k]], -sin(ϕ[i]))
@NLexpression(m, Winv31[i=1:T[k]], 0)
@NLexpression(m, Winv32[i=1:T[k]], sin(ϕ[i])/cos(θ[i]))
@NLexpression(m, Winv33[i=1:T[k]], cos(ϕ[i])/cos(θ[i]));
# Simplified
#@NLexpression(m, Winv11[i=1:T[k]], 1)
#@NLexpression(m, Winv12[i=1:T[k]], (ϕ[i])*(θ[i]))
#@NLexpression(m, Winv13[i=1:T[k]], (θ[i]))
#@NLexpression(m, Winv21[i=1:T[k]], 0)
#@NLexpression(m, Winv22[i=1:T[k]], 1)
#@NLexpression(m, Winv23[i=1:T[k]], -(ϕ[i]))
#@NLexpression(m, Winv31[i=1:T[k]], 0)
#@NLexpression(m, Winv32[i=1:T[k]], (ϕ[i]))
#@NLexpression(m, Winv33[i=1:T[k]], 1);

# Differential Transformation matrix for angular velocities from body frame to inertial frame
@NLexpression(m, dWinv11[i=1:T[k]], 0)
@NLexpression(m, dWinv12[i=1:T[k]], AVi[1,i]*cos(ϕ[i])*tan(θ[i]) + AVi[2,i]*sin(ϕ[i])/cos(θ[i])^2)
@NLexpression(m, dWinv13[i=1:T[k]], -AVi[1,i]*sin(ϕ[i])*cos(θ[i]) + AVi[2,i]*cos(ϕ[i])/cos(θ[i])^2)
@NLexpression(m, dWinv21[i=1:T[k]], 0)
@NLexpression(m, dWinv22[i=1:T[k]], -AVi[1,i]*sin(ϕ[i]))
@NLexpression(m, dWinv23[i=1:T[k]], -AVi[1,i]*cos(ϕ[i]))
@NLexpression(m, dWinv31[i=1:T[k]], 0)
@NLexpression(m, dWinv32[i=1:T[k]], AVi[1,i]*cos(ϕ[i])/cos(θ[i]) + AVi[1,i]*sin(ϕ[i])*tan(θ[i])/cos(θ[i]))
@NLexpression(m, dWinv33[i=1:T[k]], -AVi[1,i]*sin(ϕ[i])/cos(θ[i]) + AVi[2,i]*cos(ϕ[i])*tan(θ[i])/cos(θ[i]));
# Simplified
#@NLexpression(m, dWinv11[i=1:T[k]], 0)
#@NLexpression(m, dWinv12[i=1:T[k]], AVi[1,i]*(θ[i]) + AVi[2,i]*(ϕ[i]))
#@NLexpression(m, dWinv13[i=1:T[k]], -AVi[1,i]*(ϕ[i]) + AVi[2,i])
#@NLexpression(m, dWinv21[i=1:T[k]], 0)
#@NLexpression(m, dWinv22[i=1:T[k]], -AVi[1,i]*(ϕ[i]))
#@NLexpression(m, dWinv23[i=1:T[k]], -AVi[1,i])
#@NLexpression(m, dWinv31[i=1:T[k]], 0)
#@NLexpression(m, dWinv32[i=1:T[k]], AVi[1,i] + AVi[1,i]*(ϕ[i])*(θ[i]))
#@NLexpression(m, dWinv33[i=1:T[k]], -AVi[1,i]*(ϕ[i]) + AVi[2,i]*(θ[i]));

### Constraints

#### Transformation of frame of reference constraints for linear and angular velocities 

- Transformation of Angular velocities in body frame to inertial frame

In [8]:
@NLexpression(m, WnInv_νX[t=1:T[k]], AVb[1,t]*Winv11[t] + AVb[2,t]*Winv12[t] + AVb[3,t]*Winv13[t])
@NLexpression(m, WnInv_νY[t=1:T[k]], AVb[1,t]*Winv21[t] + AVb[2,t]*Winv22[t] + AVb[3,t]*Winv23[t])
@NLexpression(m, WnInv_νZ[t=1:T[k]], AVb[1,t]*Winv31[t] + AVb[2,t]*Winv32[t] + AVb[3,t]*Winv33[t])

for t in 1:T[k]
    @NLconstraint(m, AVi[1,t] == WnInv_νX[t])
    @NLconstraint(m, AVi[2,t] == WnInv_νX[t])
    @NLconstraint(m, AVi[3,t] == WnInv_νX[t])
end

- Transformation of Linear velocities in body frame to inertial frame

In [9]:
@NLexpression(m, RLVbX[t=1:T[k]], LVb[1,t]*R11[t] + LVb[2,t]*R12[t] + LVb[3,t]*R13[t])
@NLexpression(m, RLVbY[t=1:T[k]], LVb[1,t]*R21[t] + LVb[2,t]*R22[t] + LVb[3,t]*R23[t])
@NLexpression(m, RLVbZ[t=1:T[k]], LVb[1,t]*R31[t] + LVb[2,t]*R32[t] + LVb[3,t]*R33[t])

for t in 1:T[k]
    @NLconstraint(m, LVi[1,t] == RLVbX[t])
    @NLconstraint(m, LVi[2,t] == RLVbY[t])
    @NLconstraint(m, LVi[3,t] == RLVbZ[t])
end

#### Velocity constraints 

In [10]:
# satisfy the dynamics (with zero initial translational and angular velocity in body frame and inertial frame)
@constraint(m, LVb[:,1] .== [0;0;0])
@constraint(m, AVb[:,1] .== [0;0;0])
@constraint(m, LVi[:,1] .== [0;0;0])
@constraint(m, AVi[:,1] .== [0;0;0])


for t in 1:T[k]-1
    # Translational velocity and acceleration constraints in inertial frame
    @constraint(m, ξ[:,t+1] .== ξ[:,t] + LVi[:,t])
    @constraint(m, LVi[:,t+1] .== LVi[:,t] + LAi[:,t])
    
    # Angular velocity and acceleration constraints in inertial frame
    @constraint(m, η[:,t+1] .== η[:,t] + AVi[:,t])
    @constraint(m, AVi[:,t+1] .== AVi[:,t] + AAi[:,t])
end

# hit all the waypoints
for i in [1 k]
    @constraint(m, ξ[:,T[i]] .== xw[:,i])
   #@constraint(m, η[:,T[i]] .== ηw[:,i])
   # @constraint(m, LVi[:,T[i]] .== LViw[:,i])
   # @constraint(m, LAi[:,T[i]] .== LViw[:,i])
   # @constraint(m, AVi[:,T[i]] .== LViw[:,i])
   # @constraint(m, AAi[:,T[i]] .== LViw[:,i])
end

#### Newton-Euler equations
These are the constraints to be applied to obtain quadcopter dynamics
- In the body frame, the force required for the
acceleration of mass m and the centrifugal force are equal to the
gravity and the total thrust of the rotors

In [11]:
# Centrifugal Force in 3 directions
@NLexpression(m, centrifugalForceX[t=1:T[k]], (AVb[2,t]*mass*LVb[3,t] - AVb[3,t]*mass*LVb[2,t]))
@NLexpression(m, centrifugalForceY[t=1:T[k]], (AVb[3,t]*mass*LVb[1,t] - AVb[1,t]*mass*LVb[3,t]))
@NLexpression(m, centrifugalForceZ[t=1:T[k]], (AVb[1,t]*mass*LVb[2,t] - AVb[2,t]*mass*LVb[1,t]))

# Gravitational force
@NLexpression(m, RtransposeGX[t=1:T[k]], -R31[t]*mass*g)
@NLexpression(m, RtransposeGY[t=1:T[k]], -R32[t]*mass*g)
@NLexpression(m, RtransposeGZ[t=1:T[k]], -R33[t]*mass*g)

for t in 1:T[k]
    @NLconstraint(m, mass*LAb[1,t] + centrifugalForceX[t] == RtransposeGX[t])
    @NLconstraint(m, mass*LAb[2,t] + centrifugalForceY[t] == RtransposeGY[t])
    @NLconstraint(m, mass*LAb[3,t] + centrifugalForceZ[t] == RtransposeGZ[t] + TB3[t])
end

- In the inertial frame, the centrifugal force is nullified. Thus, only the gravitational
force and the magnitude and direction of the thrust are contributing in the acceleration
of the quadcopter

In [12]:
for t in 1:T[k]
    @NLconstraint(m, LAi[1,t] == -(Ax/mass)*LVi[1,t] + (TB3[t]/mass)*(cos(ψ[t])*sin(θ[t])*cos(ϕ[t]) + sin(ψ[t])*sin(ϕ[t])))
    @NLconstraint(m, LAi[2,t] == -(Ay/mass)*LVi[2,t] + (TB3[t]/mass)*(sin(ψ[t])*sin(θ[t])*cos(ϕ[t]) - cos(ψ[t])*sin(ϕ[t])))
    @NLconstraint(m, LAi[3,t] == -(Az/mass)*LVi[3,t] + (TB3[t]/mass)*(cos(θ[t])*cos(ϕ[t])) - g)
    
    # simplified
    #@NLconstraint(m, LAi[1,t] == -(Ax/mass)*LVi[1,t] + (TB3[t]/mass)*((θ[t]) + (ψ[t])*(ϕ[t])))
    #@NLconstraint(m, LAi[2,t] == -(Ay/mass)*LVi[2,t] + (TB3[t]/mass)*((ψ[t])*(θ[t]) - (ϕ[t])))
    #@NLconstraint(m, LAi[3,t] == -(Az/mass)*LVi[3,t] + (TB3[t]/mass) - g)
end

- In the body frame, the angular acceleration of the inertia , the centripetal forces and the gyroscopic forces are equal to the external torque

In [13]:
for t in 1:T[k]
    @NLconstraint(m, AAb[1,t] == (Iyy - Izz)*q[t]*r[t]/Ixx + τbϕ[t]/Ixx)
    @NLconstraint(m, AAb[2,t] == (Izz - Ixx)*p[t]*r[t]/Iyy + τbθ[t]/Iyy)
    @NLconstraint(m, AAb[3,t] == (Ixx - Iyy)*p[t]*q[t]/Izz + τbψ[t]/Izz)
end

- The angular accelerations in the inertial frame are
then attracted from the body frame accelerations with the transformation matrix
$W^{−1}$ and its time derivative

In [14]:
@NLexpression(m, dWnInv_νX[t=1:T[k]], AVb[1,t]*dWinv11[t] + AVb[2,t]*dWinv12[t] + AVb[3,t]*dWinv13[t])
@NLexpression(m, dWnInv_νY[t=1:T[k]], AVb[1,t]*dWinv21[t] + AVb[2,t]*dWinv22[t] + AVb[3,t]*dWinv23[t])
@NLexpression(m, dWnInv_νZ[t=1:T[k]], AVb[1,t]*dWinv31[t] + AVb[2,t]*dWinv32[t] + AVb[3,t]*dWinv33[t])

@NLexpression(m, WnInv_νdotX[t=1:T[k]], AAb[1,t]*Winv11[t] + AAb[2,t]*Winv12[t] + AAb[3,t]*Winv13[t])
@NLexpression(m, WnInv_νdotY[t=1:T[k]], AAb[1,t]*Winv21[t] + AAb[2,t]*Winv22[t] + AAb[3,t]*Winv23[t])
@NLexpression(m, WnInv_νdotZ[t=1:T[k]], AAb[1,t]*Winv31[t] + AAb[2,t]*Winv32[t] + AAb[3,t]*Winv33[t])

for t in 1:T[k]
    @NLconstraint(m, AAi[1,t] == dWnInv_νX[t] + WnInv_νdotX[t])
    @NLconstraint(m, AAi[2,t] == dWnInv_νY[t] + WnInv_νdotY[t])
    @NLconstraint(m, AAi[3,t] == dWnInv_νZ[t] + WnInv_νdotZ[t])
end

### Objective 

In [15]:
λ=100
@objective(m, Min, sum(ωSq) + λ*sum( (ξ[:,T[2:k]]-xw[:,2:k]).^2  ))

:Min

In [16]:
#m

In [17]:
println("Status: ",solve(m))

Bonmin 1.8.4 using Cbc 2.9.7 and Ipopt 3.12.4
bonmin: 
Cbc3007W No integer variables - nothing to do

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 41.084138      361 0.731861
Cbc3007W No integer variables - nothing to do

 	"Finished"
Status: Optimal


In [18]:
getobjectivevalue(m)

41.0841381168749

In [19]:
getvalue(ξ)

3×5 Array{Float64,2}:
 -7.62548e-25  -1.20543e-23  -0.000607601  -0.00135963  -7.54818e-25
  3.14482e-25   1.07559e-24   0.263276      0.589134     1.0        
 -5.73763e-26  -1.59526e-25   1.0           2.11357e-9   1.0        

In [20]:
getvalue(η)

3×5 Array{Float64,2}:
 -0.0243479    -0.0243479    -0.022967     -0.0128534    0.00421851 
  0.000327295   0.000327295   0.000517794   0.000732411  0.000814838
  0.0157468     0.0157468     0.0159373     0.0161519    0.0162344  

In [21]:
getvalue(ωSq)

4×5 Array{Float64,2}:
  1.85746  9.62158e-6  1.36465e-7  0.817466  1.70256
  0.0      1.99235     2.91695e-7  1.19757   1.70256
  3.20312  1.91388     5.27847     1.89037   1.09754
 10.0      9.62158e-6  2.74661e-7  1.72937   1.09753

In [22]:
getvalue(LAi)

3×5 Array{Float64,2}:
 -0.000607601  -0.000144431  0.00211166   0.00365194   0.00374897
  0.263276      0.0625825    0.0850083   -0.112138    -0.19017   
  1.0          -2.0          2.0         -2.0         -2.0       

In [23]:
getvalue(LVi)

3×5 Array{Float64,2}:
 -1.05292e-23  -0.000607601  -0.000752032  0.00135963   0.00501157
  4.46629e-25   0.263276      0.325858     0.410866     0.298729  
 -4.47729e-26   1.0          -1.0          1.0         -1.0       

In [24]:
getvalue(AVi)

3×5 Array{Float64,2}:
 -8.06427e-24  0.00138083   0.0101136    0.0170719    0.0187712 
  2.71818e-22  0.000190499  0.000214617  8.24271e-5  -0.00013958
 -5.40348e-23  0.000190499  0.000214617  8.24271e-5  -0.00013958

In [25]:
getvalue(AAi)

3×5 Array{Float64,2}:
 0.00138083   0.00873278   0.00695835   0.0016992    0.00386925 
 0.000190499  2.41175e-5  -0.00013219  -0.000222007  0.000498137
 0.000190499  2.41175e-5  -0.00013219  -0.000222007  0.000602359